In [3]:
# Cuckoo Search implementation for a simple electrical grid economic dispatch problem.
# This code is visible to the user and will run here. It finds generator outputs (Pg)
# that minimize total generation cost while meeting demand (with penalty for constraint violations).
# Uses Lévy flights for generating new solutions.

import numpy as np
import math
import random
from dataclasses import dataclass

@dataclass
class Generator:
    Pmin: float
    Pmax: float
    a: float 
    b: float 
    c: float 

def total_cost(Pg, gens: list[Generator]):
    # quadratic cost: a*P^2 + b*P + c
    cost = 0.0
    for p, g in zip(Pg, gens):
        cost += g.a * p * p + g.b * p + g.c
    return cost

def levy_flight(beta=1.5):
    # Mantegna's algorithm for Levy stable distribution step
    sigma_u = (math.gamma(1 + beta) * math.sin(math.pi * beta / 2) /
               (math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
    u = np.random.normal(0, sigma_u)
    v = np.random.normal(0, 1)
    step = u / (abs(v) ** (1 / beta))
    return step

def simple_bounds(Pg, gens):
    # ensure generator outputs within bounds
    Pg_bounded = np.copy(Pg)
    for i, g in enumerate(gens):
        Pg_bounded[i] = np.clip(Pg_bounded[i], g.Pmin, g.Pmax)
    return Pg_bounded

def fitness(Pg, gens, demand, penalty_factor=1e5):
    # objective = cost + large penalty for power imbalance
    cost = total_cost(Pg, gens)
    imbalance = abs(np.sum(Pg) - demand)
    return cost + penalty_factor * imbalance

def cuckoo_search(gens, demand, n_nests=25, max_iter=500, pa=0.25, beta=1.5, verbose=False):
    dim = len(gens)
    # initialize nests (solutions) uniformly between Pmin and Pmax
    nests = np.zeros((n_nests, dim))
    for i in range(n_nests):
        for j, g in enumerate(gens):
            nests[i, j] = np.random.uniform(g.Pmin, g.Pmax)
    # evaluate
    fitnesses = np.array([fitness(nests[i], gens, demand) for i in range(n_nests)])
    # best
    best_idx = np.argmin(fitnesses)
    best_nest = nests[best_idx].copy()
    best_fit = fitnesses[best_idx]
    if verbose:
        print(f"Initial best cost (with penalty): {best_fit:.6f}")
    for it in range(max_iter):
        # generate new solutions by Lévy flights from a randomly chosen nest
        for i in range(n_nests):
            step = levy_flight(beta)
            # scale step by difference between current and best
            step_size = 0.01 * step * (nests[i] - best_nest)
            # random walk
            new_nest = nests[i] + step_size * np.random.randn(dim)
            new_nest = simple_bounds(new_nest, gens)
            new_fit = fitness(new_nest, gens, demand)
            # greedy selection
            if new_fit < fitnesses[i]:
                nests[i] = new_nest
                fitnesses[i] = new_fit
                if new_fit < best_fit:
                    best_fit = new_fit
                    best_nest = new_nest.copy()
        # discovery and randomization (some nests get replaced)
        K = np.random.rand(n_nests) < pa
        for i in range(n_nests):
            if K[i]:
                # replace with a new random solution (or use two random nests difference)
                idx1, idx2 = np.random.choice(n_nests, 2, replace=False)
                step = np.random.rand(dim) * (nests[idx1] - nests[idx2])
                new_nest = nests[i] + step
                new_nest = simple_bounds(new_nest, gens)
                new_fit = fitness(new_nest, gens, demand)
                if new_fit < fitnesses[i]:
                    nests[i] = new_nest
                    fitnesses[i] = new_fit
                    if new_fit < best_fit:
                        best_fit = new_fit
                        best_nest = new_nest.copy()
        if verbose and (it % (max_iter//10 + 1) == 0):
            print(f"Iter {it}/{max_iter} best cost: {best_fit:.6f}")
    # return best solution and its true cost and imbalance
    imbalance = np.sum(best_nest) - demand
    true_cost = total_cost(best_nest, gens)
    return {
        "Pg": best_nest,
        "cost_with_penalty": best_fit,
        "true_cost": true_cost,
        "imbalance": imbalance,
        "fitnesses": fitnesses
    }

# Example: 3-generator system (classic economic dispatch)
gens = [
    Generator(Pmin=50, Pmax=200, a=0.002, b=8.0, c=100.0),
    Generator(Pmin=50, Pmax=150, a=0.003, b=6.5, c=120.0),
    Generator(Pmin=40, Pmax=100, a=0.0015, b=9.0, c=80.0),
]

demand = 300.0  # MW

result = cuckoo_search(gens, demand, n_nests=40, max_iter=800, pa=0.25, beta=1.5, verbose=True)

print("\n=== Best solution found ===")
for i, p in enumerate(result["Pg"], start=1):
    print(f"Generator {i}: P = {p:.4f} MW (limits: {gens[i-1].Pmin}-{gens[i-1].Pmax})")
print(f"Total generation = {np.sum(result['Pg']):.4f} MW, Demand = {demand} MW, Imbalance = {result['imbalance']:.6f} MW")
print(f"True generation cost (no penalty) = {result['true_cost']:.4f} monetary units")



print()
print()
print()
print("Executed by Poorvi Naveen (1BM23CS234)")


Initial best cost (with penalty): 44774.680584
Iter 0/800 best cost: 44774.680584
Iter 81/800 best cost: 2662.500000
Iter 162/800 best cost: 2617.719280
Iter 243/800 best cost: 2617.719280
Iter 324/800 best cost: 2617.719280
Iter 405/800 best cost: 2617.719280
Iter 486/800 best cost: 2617.336329
Iter 567/800 best cost: 2617.336329
Iter 648/800 best cost: 2617.336329
Iter 729/800 best cost: 2617.336329

=== Best solution found ===
Generator 1: P = 110.1193 MW (limits: 50-200)
Generator 2: P = 149.8808 MW (limits: 50-150)
Generator 3: P = 40.0000 MW (limits: 40-100)
Total generation = 300.0001 MW, Demand = 300.0 MW, Imbalance = 0.000081 MW
True generation cost (no penalty) = 2609.2247 monetary units



Executed by Poorvi Naveen (1BM23CS234)
